In [ ]:
"""mc_sphere_qrng.ipynb"""
# Cell 1

from __future__ import annotations

import typing

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.markers import MarkerStyle
from numba import float64, vectorize  # type: ignore
from numpy.random import default_rng

if typing.TYPE_CHECKING:
    from matplotlib.axes import Axes
    from numpy.random import Generator
    from numpy.typing import NDArray

%matplotlib widget


iterations_cube: int = 50
iterations: int = iterations_cube**3


def calc_prng() -> None:
    rng: Generator = default_rng(seed=2020)
    x: NDArray[np.float_] = rng.random(iterations) * 2 - 1
    y: NDArray[np.float_] = rng.random(iterations) * 2 - 1
    z: NDArray[np.float_] = rng.random(iterations) * 2 - 1

    d: NDArray[np.float_] = x**2 + y**2 + z**2

    est_volume: float = np.count_nonzero(d <= 1.0) / iterations * 8
    act_volume: float = 4.0 / 3.0 * np.pi
    err: float = (est_volume - act_volume) / act_volume

    print(
        (
            f"Total dots       = {iterations:,}\n"
            f"Act. Volume      = {act_volume:.6f}\n"
            f"PRNG Est. Volume = {est_volume:.6f}\n"
            f"PRNG % Rel Err   = {err:.6%}\n"
        )
    )


@vectorize([float64(float64, float64)], nopython=True)  # type: ignore
def halton(n: float, p: int) -> float:
    h = 0
    f = 1
    while n > 0:
        f: float = f / p
        h += (n % p) * f
        n = int(n / p)
    return h


def plot_qrng(ax: Axes) -> None:
    primes: list[int] = [2, 3, 5]

    x: NDArray[np.float_] = halton(np.arange(iterations), primes[0]) * 2 - 1  # type: ignore
    y: NDArray[np.float_] = halton(np.arange(iterations), primes[1]) * 2 - 1  # type: ignore
    z: NDArray[np.float_] = halton(np.arange(iterations), primes[2]) * 2 - 1  # type: ignore

    d: NDArray[np.float_] = x**2 + y**2 + z**2

    x_in: NDArray[np.float_] = x[d <= 1.0]
    y_in: NDArray[np.float_] = y[d <= 1.0]
    z_in: NDArray[np.float_] = z[d <= 1.0]

    x_out: NDArray[np.float_] = x[d > 1.0]
    y_out: NDArray[np.float_] = y[d > 1.0]
    z_out: NDArray[np.float_] = z[d > 1.0]

    ax.set_title("Sphere Volume via Monte Carlo Estimation (Halton QRNG)")
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")  # type: ignore
    ax.view_init(azim=-72, elev=48)  # type: ignore

    pixel_size: float = (72 / ax.figure.dpi) ** 2  # type: ignore
    ax.scatter(x_in, y_in, z_in, color="red", marker=MarkerStyle("."), s=pixel_size)
    ax.scatter(x_out, y_out, z_out, color="blue", marker=MarkerStyle("."), s=pixel_size)

    est_volume: float = len(x_in) / iterations * 8
    act_volume: float = 4.0 / 3.0 * np.pi
    err: float = (est_volume - act_volume) / act_volume

    print(
        (
            f"Total dots       = {iterations:,}\n"
            f"Act. Volume      = {act_volume:.6f}\n"
            f"QRNG Est. Volume = {est_volume:.6f}\n"
            f"QRNG % Rel Err   = {err:.6%}"
        )
    )

    ax.figure.set_size_inches(11, 8.5)


def main() -> None:
    calc_prng()

    plt.close("all")
    plt.figure(" ", figsize=(11, 6), constrained_layout=True)
    plot_qrng(plt.axes(projection="3d"))
    plt.show()


main()